### Tool Calling

In [1]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv
load_dotenv()


True

### Tool Creation

In [2]:
@tool
def multiply(a: int , b: int) -> int:
    """Given 2 numbers a and b, this tool returns their product"""
    return a * b


In [3]:
print(multiply.invoke({'a': 3, 'b': 8}))

24


In [4]:
multiply.name

'multiply'

In [5]:
multiply.description

'Given 2 numbers a and b, this tool returns their product'

In [6]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

### Tool Binding

In [7]:
model = ChatGroq(model='llama-3.1-8b-instant')

In [8]:
model.invoke('Hi')

AIMessage(content="It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 36, 'total_tokens': 59, 'completion_time': 0.024107141, 'prompt_time': 0.041124059, 'queue_time': 0.045924701, 'total_time': 0.0652312}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--8407fa92-86bd-4804-8fb5-bc9741e847e5-0', usage_metadata={'input_tokens': 36, 'output_tokens': 23, 'total_tokens': 59})

In [9]:
llm_with_tools = model.bind_tools([multiply])

In [10]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm functioning properly, thanks for asking. Is there anything I can help you with?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 227, 'total_tokens': 246, 'completion_time': 0.032573618, 'prompt_time': 0.022893977, 'queue_time': 0.049038799, 'total_time': 0.055467595}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--10d4171c-a6e0-4d22-a5a7-73e53ac25c34-0', usage_metadata={'input_tokens': 227, 'output_tokens': 19, 'total_tokens': 246})

In [11]:
query = HumanMessage('can you multiply 3 with 1000')

In [12]:
messages = [query]

In [13]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [14]:
result = llm_with_tools.invoke(messages)

In [15]:
messages.append(result)

In [16]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'he5mkn51t', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 232, 'total_tokens': 252, 'completion_time': 0.029674808, 'prompt_time': 0.745317369, 'queue_time': 0.36970475, 'total_time': 0.774992177}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ca9d93d1-a81e-4bb2-821f-1a6324125753-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'he5mkn51t', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 20, 'total_tokens': 252})]

In [17]:
tool_result = multiply.invoke(result.tool_calls[0])

In [18]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='he5mkn51t')

In [19]:
messages.append(tool_result)

In [20]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'he5mkn51t', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 232, 'total_tokens': 252, 'completion_time': 0.029674808, 'prompt_time': 0.745317369, 'queue_time': 0.36970475, 'total_time': 0.774992177}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ca9d93d1-a81e-4bb2-821f-1a6324125753-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'he5mkn51t', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 20, 'total_tokens': 252}),
 ToolMessage(content='3000', name='multiply', tool_call_id='he5mkn51t')]

In [22]:
llm_with_tools.invoke(messages).content

'The result of the multiplication is 3000.'

### Currency Converter Using Langchain Tool

In [23]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [24]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

In [25]:
@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate

In [26]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [27]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1754352001,
 'time_last_update_utc': 'Tue, 05 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1754438401,
 'time_next_update_utc': 'Wed, 06 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.7131}

In [31]:
convert.invoke({'base_currency_value':3000, 'conversion_rate':86})

258000.0

In [32]:
# tool binding
llm = ChatGroq(model='llama-3.1-8b-instant')

In [37]:
llm_model_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [38]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [39]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [40]:
ai_message = llm_model_with_tools.invoke(messages)

In [41]:
messages.append(ai_message)

In [42]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'ke412babh',
  'type': 'tool_call'}]

In [43]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [44]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'ke412babh', 'function': {'arguments': '{"base_currency":"INR","target_currency":"USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 341, 'total_tokens': 388, 'completion_time': 0.090002018, 'prompt_time': 1.047860975, 'queue_time': 0.046442643, 'total_time': 1.137862993}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--655749e6-47a6-48d8-9f76-ded21c056c9a-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': 'ke412babh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 341, 'output_tok

In [52]:
llm_model_with_tools.invoke(messages).content

''

In [50]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\subhadip\AppData\Local\Temp\ipykernel_11492\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [51]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Action:
{
  "action": "Final Answer",
  "action_input": "I'm a large language model, so I don't have emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you have."
}

> Finished chain.
